# Основы анализа данных в Python

*Тамбовцева Алла, НИУ ВШЭ*

## Повторение: доверительные интервалы, проверка гипотез, коэффициенты корреляции

### Информация о данных

В рамках этого файла используем данные с результатами опроса преподавателей двух каталонских универсистетов (Открытый университет Каталонии и университет Помпеу Фабра), посвященного использованию Википедии в методических и академических целях.

* Codebook для набора данных: [ссылка](https://github.com/allatambov/PyDataAnalysis/blob/main/wiki_codebook.pdf).
* Оригинальное описание на платформе data.world: [ссылка](https://data.world/uci/wiki4he).

### Загрузка данных 

Импортируем библиотеку `pandas` с сокращенным названием `pd` (понадобится для работы с данными) и библиотеку `numpy` с сокращенным названием `np` (понадобится для более удобных вычислений):

In [1]:
import numpy as np
import pandas as pd

Загрузим данные из файла `"wiki.csv"` и сохраним их в датафрейм `wiki`, сообщив Python, что в качестве разделителя столбцов используется точка с запятой, а не запятая:

In [2]:
wiki = pd.read_csv("wiki.csv", sep = ";")

Если мы не добавим аргумент `sep` с разделителем, все данные склеятся в один столбец, так как по умолчанию функция `read_csv()` подразумевает разделение по запятой (ведь название формата CSV – это сокращение от *comma separated values*).

**Примечание для Jupyter Notebook.** Удобно, если файл с данными при работе лежит в той же папке, что и текущий ipynb-файл, в котором мы запускаем код, так не придется полностью прописывать к нему путь, достаточно одного названия с расширением.

**Примечание для Google Colab.** Загрузить файл с данными в облачное хранилище можно через кнопку *Files* (значок папки слева от рабочей области с ячейками), при нажатии на которую появляется возможность выбрать файл с компьютера (значок стрелки). После добавления файла его можно выбрать, кликнуть на три точки справа от названия, скопировать путь через *Copy path* и вставить его в функцию `read_csv()`. Например:

    wiki = pd.read_csv("/content/wiki.csv", sep = ";") 

Определим размерность датафрейма – число строк и число столбцов:

In [3]:
wiki.shape

(913, 53)

Итак, в датафрейме 913 строк и 53 столбца. Значит, всего было опрошено 913 человек.

### Работа с долями: бинарные переменные

Выберем столбец `GENDER`, который соответствует полу респондента (0 – мужской, 1 – женский), укажем название этого столбца в квадратных скобках:

In [4]:
wiki["GENDER"]

0      0
1      0
2      0
3      0
4      0
      ..
908    0
909    0
910    0
911    0
912    1
Name: GENDER, Length: 913, dtype: int64

Проверим, что в этом столбце нет значений, кроме 0 и 1. Запросим уникальные значения с помощью метода `.unique()`:

In [5]:
wiki["GENDER"].unique()

array([0, 1])

Все нормально, лишнего ничего нет. Посчитаем, сколько в столбце нулей и единиц. Для этого сформируем таблицу частот с помощью метода `.value_counts()`:

In [6]:
wiki["GENDER"].value_counts()

0    525
1    388
Name: GENDER, dtype: int64

Итак, в опросе участвовало 525 женщин и 388 мужчин. Допустим, нас интересует доля мужчин, принявших участие в опросе. В нашем случае эта доля примерно соответствует доле мужчин, работающих в двух университетах Каталонии (Открытом университете и университете Помпеу-Фабра). 

Понятно, что чтобы определить такую долю, нам нужно 388 поделить на общее число респондентов, а их у нас 913 (общее число строк в датафрейме). 

Вопрос: а как посчитать долю более универсальным образом? Ответ: автоматически посчитать число единиц, автоматически посчитать общее число элементов в столбце и поделить одно на другое. Если у нас есть набор из 0 и 1, число единиц совпадает с суммой элементов (сумма нулей ничего нового не дает). Посчитаем сумму элементов столбца через метод `.sum()`:

In [7]:
wiki["GENDER"].sum()

388

Общее число элементов в столбце хранится в атрибуте `.size`:

In [8]:
wiki["GENDER"].size

913

*Пояснение.* Атрибут – некоторая фиксированная характеристика объекта (число строк или столбцов в датафрейме, названия строк или столбцов, тип данных), метод – функция, которая выполняет некоторую операцию над объектом. И атрибуты, и методы вызываются через точку, но в конце метода всегда есть круглые скобки (например, `.sum()`), они означают, что мы хотим применить метод, а не просто вызвать и посмотреть на него.

Для вычисления доли поделим одно на другое:

In [9]:
wiki["GENDER"].sum() / wiki["GENDER"].size

0.4249726177437021

Несложно догадаться, что мы сейчас просто посчитали среднее арифметическое столбца `GENDER`. Конечно, можно было поступить проще — воспользоваться методом `.mean()`. 

In [10]:
wiki["GENDER"].mean()

0.4249726177437021

Но наши вычисления нам тоже сейчас пригодятся.

### Доверительный интервал для доли

Вспомним, как вычисляется 95%-ный доверительный интервал для доли (примерно):

$$
\hat{p} - 2 \times s_{\hat{p}} < p < \hat{p} + 2 \times s_{\hat{p}}.
$$

Здесь $p$ – доля в генеральной совокупности, которая нам неизвестна и которую мы хотим оценить с помощью доверительного интервала, $\hat{p}$ – доля, полученная по выборке, $s_{\hat{p}}$ – стандартное отклонение доли, тоже полученное по выборке, которое вычисляется так ($n$ – число наблюдений в выборке):

$$
s_{\hat{p}} = \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}.
$$

Давайте используем строчки кода, написанные ранее, для вычисления характеристик, которые присутствуют в формулах, и построим 95%-ный доверительный интервал для доли мужчин среди профессорско-преподавательского состава двух университетов Каталонии.

In [11]:
n = wiki["GENDER"].size  # размер выборки
p = wiki["GENDER"].sum() / n  # доля p^

Теперь, зная $n$ и $\hat{p}$, мы можем посчитать стандартное отклонение доли $s_{\hat{p}}$. Для извлечения квадратного корня возьмем функцию `sqrt()` из библиотеки `numpy`:

In [12]:
s_p = np.sqrt(p * (1 - p) / n) 
s_p

0.016360227864035466

Отлично! Осталось все собрать воедино и посчитать границы доверительного интервала! Подставим все в формулу:

In [13]:
(p - 2 * s_p, p + 2 * s_p) 

(0.3922521620156312, 0.457693073471773)

Итак, можем сказать, что с 95% уверенностью можно считать, что истинная доля мужчин, работающих в университетах Каталонии, лежит в интервале от 0.39 до 0.46.

Тот же самый интервал мы могли бы посчитать, воспользовавшись специальной функцией из модуля `stats` библиотеки `scipy` (от *scientific python*). Этот модуль часто импортируется с сокращенным названием:

In [14]:
import scipy.stats as st

Сама функция называется `norm.interval()` позволяет строить некоторые интервалы, предполагая, что те оценки, для которых мы эти интервалы строим, берутся из нормального распределения с некоторым средним (`loc`) и некоторым стандартным отклонением (`scale`). Можем подставить наши значения `p` и `s_p` в качестве параметров такого распределения и зафиксировать уровень доверия:

In [15]:
st.norm.interval(0.95, loc = p, scale = s_p)

(0.39290716035132395, 0.45703807513608025)

Данный интервал чуть более точный, чем тот, который мы посчитали ранее, потому что значение, на которое мы домножаем `s_p`, на самом деле, не совсем 2, а чуть поменьше (1.96, если вспомнить курс по теории вероятностей).

### Доверительный интервал для среднего

Допустим, мы хотим понять, в каком интервале, с 95%-ной уверенностью, лежит среднее значение возраста сотрудников двух каталонских университетов. 

Для начала посмотрим на описательные статистики столбца `AGE`:

In [16]:
wiki["AGE"].describe()

count    913.000000
mean      42.246440
std        8.058418
min       23.000000
25%       36.000000
50%       42.000000
75%       47.000000
max       69.000000
Name: AGE, dtype: float64

Итак, средний возраст примерно равен 42 годам, медианный тоже. Но, на самом деле, истинное среднее значение возраста может быть немного иным, исследователи же не смогли опросить всех сотрудников обоих университетов, их там много!

Чтобы учесть такую неопределенность в наших оценках, построим 95%-ный доверительный интервал для среднего. Вспомним, как он считается:

$$
\hat{\mu} - 2 \times s_{\hat{\mu}} < \mu < \hat{\mu} + 2 \times s_{\hat{\mu}}.
$$

Здесь $\mu$ – среднее генеральной совокупности, которое нам неизвестно и которое мы хотим оценить с помощью доверительного интервала, $\hat{\mu}$ – среднее, полученное по выборке, $s_{\hat{\mu}}$ – стандартное отклонение среднего, тоже полученное по выборке, которое вычисляется на основе стандартного отклонения самой выборки $s$ и ее объема $n$:

$$
s_{\hat{\mu}} = \frac{s}{\sqrt{n}}
$$


Итак, опять соберем все составные части для формулы. 

In [17]:
n = wiki["AGE"].size  # размер выборки
mu = wiki["AGE"].mean()  # среднее выборки mu^
s = wiki["AGE"].std()  # ст отклонение выборки s

Вычислим стандартное отклонение среднего:

In [18]:
s_mu = s / np.sqrt(n)
s_mu

0.26669471758704505

Для построения доверительного интервала для среднего также воспользуемся специальной функцией из модуля `stats`. В случае с долей мы использовали функцию `norm.interval()`, так как там нам нужно было нормальное распределение, а здесь нам понадобится функция `t.interval()` для распределения Стьюдента. Помимо уровня доверия, значение `mu` и `s_mu` нам необходимо число степеней свободы для распределения Стьюдента, оно равно $\text{df} = n - 1$. Подставлям все и вычисляем:

In [19]:
st.t.interval(0.95, df = n - 1, loc = mu, scale = s_mu) 

(41.723033639899846, 42.7698469734627)

Итак, доверительный интервал получился довольно узким. С 95%-ной уверенностью можно утверждать, что средний возраст сотрудников каталонских университетов лежит в интервале от 42 до 43 лет (тут разумнее округлить значения до целых).

### Проверка гипотез: сравнение средних двух групп

Допустим, мы хотим выяснить, отличается ли средний возраст сотрудников каталонских университетов, которые считают использование Википедии полезным для преподавания, и тех сотрудников, которые так не считают.

Полезность Википедии для преподавания участники опроса оценивали в вопросе `QU3`. Они выражали свою степень согласия с утверждением *Wikipedia is useful for teaching*. Посмотрим на ответы:

In [20]:
wiki["PU3"].value_counts()

3    312
4    250
5    175
2    151
1     20
?      5
Name: PU3, dtype: int64

Давайте создадим столбец, который будет состоять из значений `USEFUL`, если респондент считает Википедию полезной (высокая степень согласия с утверждением, значения 4 или 5 по шкале Лайкерта), и значений `USELESS`, если респондент так не считает (низкая степень согласия с утверждением, значения 1 или 2 по шкале Лайкерта). Пропущенные ответы `?` и категорию *Neither disagree nor agree* проигнорируем, потом исключим.

Как удобнее всего это сделать, чтобы избежать циклов для пробегания по всем ячейкам столбца `PU3`? Написать функцию, которая принимает на вход значение *одной ячейки* и возвращает ответ `USEFUL` или `USELESS`, а затем применить ее ко всему столбцу с помощью специального метода.

Напишем функцию под названием `check_use()`, на вход она принимает значение `x`, проверяет условия, и, в зависимости от их выполнения, возвращает строку с тем или иным ответом:

In [21]:
def check_use(x):
    
    # если 4 или 5 возвращаем USEFUL
    # если иначе, тестим следующее условие для 1 и 2
    # если и оно не выполняется, ставим пропуск – пустое значение
    
    if (x == '4') | (x == '5'):
        r = 'USEFUL'
    elif (x == '1') | (x == '2'):
        r = 'USELESS'
    else:
        r = None
    return r

Проверим работу функции на отдельных значениях:

In [22]:
check_use('4')

'USEFUL'

In [23]:
check_use('1')

'USELESS'

In [24]:
check_use('?') # None 

Теперь применим функцию ко всем ячейкам в столбце, используя метод `.apply()`, и сохраним значения в новый столбец `USEFULNESS`:

In [25]:
wiki["USEFULNESS"] = wiki["PU3"].apply(check_use)

Проверим, что получилось:

In [26]:
wiki["USEFULNESS"].value_counts()

USEFUL     425
USELESS    171
Name: USEFULNESS, dtype: int64

Все прекрасно, но метод `.value_counts()` игнорирует пропуски, не показывает, сколько их. Воспользуемся методом `.isna()`:

In [27]:
wiki["USEFULNESS"].isna()

0       True
1       True
2      False
3      False
4      False
       ...  
908     True
909    False
910     True
911    False
912     True
Name: USEFULNESS, Length: 913, dtype: bool

Этот метод для каждой ячейки в столбце возвращает значение `True` (пропуск есть) или `False` (пропуска нет). Чтобы посчитать количество пропусков, нужно просто посчитать сумму по этому набору `True` и `False`, так как для Python значение `True` равносильно 1, а значение `False` равносильно 0. А про суммирование набора из нулей и единиц мы уже поговорили:

In [28]:
wiki["USEFULNESS"].isna().sum()

317

Итого, 317 строк с пропущенными значениями в столбце `USEFULNESS`. Давайте их удалим, но сделаем это осторожно. Если мы применим метод `.dropna()` для удаления строк с пропущенными значениями ко всему датафрейму, удалятся все строки, где есть хотя бы одно пропущенное значение (не только в столбце `USEFULNESS`). В нашем случае это не так страшно, потому что «чистых» пропусков в датафрейме нет, (кроме тех, которые мы сами создали, добавив `None`), отсутствие ответов закодировано `?`. Но вообще это может быть большой проблемой, особенно, если набор данных большой. Если столбцов много, есть риск, что не все они заполнены полностью, то есть, теоретически, в каждой строке будет хотя бы один пропуск. Если использовать `.dropna()` бездумно, можно совсем без данных остаться!

Поэтому мы поступим аккуратно (хоть в конкретном случае это не требуется), удалим строки с пропусками в столбце `USEFULNESS`:

In [29]:
# в suubset можно вписать целый список названий столбцов

wiki = wiki.dropna(subset = ["USEFULNESS"])

Ура! Наконец-то все готово для сравнения средних. Для начала сравним выборочные средние. Сгруппируем данные по столбцу `USEFULNESS` и посчитаем средние:

In [30]:
wiki.groupby("USEFULNESS").mean()

,AGE,GENDER,PhD,UNIVERSITY
USEFULNESS,,,,
USEFUL,42.197647,0.327059,0.451765,1.131765
USELESS,43.011696,0.491228,0.456140,1.070175


По умолчанию функция `.mean()` вычисляет средние для всех числовых столбцов. Если нам это не нравится, можем выбрать только один нужный (обычный выбор столбца по названию из маленького датафрейма выше):

In [31]:
wiki.groupby("USEFULNESS").mean()["AGE"]

USEFULNESS
USEFUL     42.197647
USELESS    43.011696
Name: AGE, dtype: float64

Судя по всему, различия в средних небольшие. Но не помешает проверить более строго, вдруг эта разница в средних в один год все-таки существенна из-за небольшого разброса данных? 

Для проверки гипотезы о равенстве средних в двух группах с помощью t-критерия Стьюдента можно воспользоваться функцией `ttest_ind` все из того же модуля `stats`. Часть с `_ind` отвечает за *independent samples*, независимые выборки (например, разные люди в двух выборках, а не одни и те же в начале исследования и в конце). Эта функция всем хороша, но есть небольшой минус: она принимает на вход две отдельные выборки, поэтому их сначала надо будет получить, отфильтровать соответствующие строки.

In [32]:
wiki_useful = wiki[wiki["USEFULNESS"] == "USEFUL"]
wiki_useless = wiki[wiki["USEFULNESS"] == "USELESS"]

Как устроен код выше? В квадратных скобках, как обычно, указывается некоторый критерий отбора, здесь он представлен в виде условия (помним, что оператор `==` используется для проверки равенства, а оператор `=` для присваивания, второй нам здесь не подойдет). Выражение в квадратных скобках, само по себе, проверяет условие для каждой строки в `wiki` и возвращает набор из логических значений `True` (если условие выполнено) и `False` (если условие не выполнено):

In [33]:
wiki["USEFULNESS"] == "USEFUL"

2      False
3       True
4       True
6       True
7       True
       ...  
903     True
904     True
907    False
909     True
911     True
Name: USEFULNESS, Length: 596, dtype: bool

Соответственно, строка кода в ячейке выше отбирает те строки из `wiki`, где выражение в квадратных скобках вернуло `True`. 

Все, теперь точно все готово для проверки гипотезы. Проверяем гипотезу о равенстве среднего возраста тех сотрудников, которые считают Википедию полезной для преподавания и тех, кто считает ее бесполезной:

In [34]:
# в скобках – две выборки, 
# столбец AGE из одной группы и из другой

st.ttest_ind(wiki_useful["AGE"], wiki_useless["AGE"])

Ttest_indResult(statistic=-1.1245735952953264, pvalue=0.2612239945157787)

Функция вернула нам два значения: значение t-статистики и p-value. Если мы примем уровень значимости равным 5% (или уровень доверия 95%), у нас не будет оснований отвергнуть нулевую гипотезу (p-value > 0.05). Данные не позволяют выявить различия в средних, можно считать средние значения возраста сотрудников, считающих Википедию полезной и бесполезной, равными.

Важное замечание: по умолчанию `ttest_ind()` использует двустроннюю альтернативу. Если мы хотим выбрать левостороннюю (первое среднее меньше второго) или правосторонюю (первое среднее больше второго), понадобится аргумент `alternative`, равный `"less"` или `"right"`. Правда, этот аргумент доступен только с версии `stats` 1.7.0, поэтому, если код ниже не работает, можно просто поделить p-value из предыдущего результата на 2, пользуясь тем, что распределение симметрично.

In [36]:
st.ttest_ind(wiki_useful["AGE"], wiki_useless["AGE"], 
            alternative = "left")

### Коэффициенты корреляции

В завершение работы с данными давайте посмотрим на то, как вычислять в Python коэффициенты корреляции Пирсона и Спирмена. Это довольно просто.

Коэффициент Пирсона можно вычислить несколькими способами. Например, воспользоваться методом `.corr()` от библиотеки `pandas`. Для этого надо выбрать нужные числовые столбцы и применить к ним этот метод. Выберем столбцы с возрастом респондента и числом лет опыта работы:

In [37]:
wiki[["AGE", "YEARSEXP"]].corr()

,AGE
AGE,1.0


Что-то пошло не так. Python показал нам только корреляцию `AGE` с `AGE`, коэффициент равный 1. Почему? Потому что, к сожалению, как это бывает при работе с реальными данными, столбец `YEARSEXP` оказался текстовым, хотя по логике должен состоять из чисел. Посмотрим, что не так с `YEARSEXP`. Выведем уникальные значения:

In [38]:
wiki["YEARSEXP"].unique() 

array(['13', '8', '11', '12', '14', '25', '15', '18', '20', '5', '10',
       '9', '6', '?', '7', '36', '4', '2', '17', '23', '16', '3', '21',
       '22', '1', '24', '27', '26', '19', '30', '28', '37', '43', '35',
       '39', '0'], dtype=object)

Ага! Проблема в знаке вопроса – так в этих данных были закодированы пропущенные ответы. Выкинем строки с пропущенными ответами – выберем только те строки, где значение в столбце `YEARSEXP` не `?`:

In [39]:
wiki = wiki[wiki["YEARSEXP"] != "?"] 

Проверим:

In [40]:
wiki["YEARSEXP"].unique() 

array(['13', '8', '11', '12', '14', '25', '15', '18', '20', '5', '10',
       '9', '6', '7', '36', '4', '2', '17', '23', '16', '3', '21', '22',
       '1', '24', '27', '26', '19', '30', '28', '37', '43', '35', '39',
       '0'], dtype=object)

Лишнее выкинулось, но небольшая проблема осталась. Значения по-прежнему текстовые! Первое, что хочется сделать – пройтись в цикле по всем элементам массива выше и преобразовать каждый в число. Но, к счастью, есть более быстрый и компактный способ – воспользоваться методом `.astype()`, он умеет изменять тип сразу всех элементов массива или столбца:

In [41]:
wiki["YEARSEXP"] = wiki["YEARSEXP"].astype(int) # переделываем в integer

Теперь метод `.corr()` работает как надо:

In [42]:
wiki[["AGE", "YEARSEXP"]].corr()

,AGE,YEARSEXP
AGE,1.000000,0.559238
YEARSEXP,0.559238,1.000000


Мы получили корреляционную матрицу, в ней сохранены коэффициенты для пары `AGE` и `YEARSEXP` и `YEARSEXP` и `AGE`. Коэффициенты одинаковы, нет разницы, смотрим ли мы на связь первого со вторым или второго с первым. Связь положительная (что ожидаемо для возраста и опыта работы), умеренная. 

Если нас не интересует корреляционная матрица, а интересует только одно значение коэффициента Пирсона, можем воспользоваться функцией `pearsonr()` из модуля `stats`:

In [43]:
# в скобках оба столбца через запятую

st.pearsonr(wiki["AGE"], wiki["YEARSEXP"])

(0.5592377622336536, 7.276531536556025e-49)

Эта функция возвращает кортеж из двух чисел: значение коэффициента Пирсона и p-value, полученное по итогам проверки значимости этого коэффициента. Здесь p-value практически 0, следовательно, гипотезу о равенстве нулю истинного коэффициента корреляции стоит отвергнуть (на любом разумном уровне значимости или уровне доверия), показатели связаны. 

Для коэффициента Спирмена все аналогично, только в `.corr()` добавляется  аргумент с указанием типа коэффициента:

In [44]:
wiki[["AGE", "YEARSEXP"]].corr(method = "spearman")

,AGE,YEARSEXP
AGE,1.000000,0.502484
YEARSEXP,0.502484,1.000000


А в `stats` используется функция с соответствующим названием:

In [45]:
st.spearmanr(wiki["AGE"], wiki["YEARSEXP"])

SpearmanrResult(correlation=0.5024836891059221, pvalue=2.629924812320569e-38)

Выводы те же, связь есть (или согласованность рангов, если точнее).